## Bayesian Optimization Process
### In this stage, we use bayesian optimization to find the best CVD hyperparameters for the ReSe2 dendritic.

### Load the experimental data

In [ ]:
import numpy as np
import pandas as pd
import emukit
import GPy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

#c−Al2O3 substrate is represented as 1, and MgO is represented as 0.
df_rese2 = pd.read_excel('Dataset_bayesian.xlsx',sheet_name='initial')
df_rese2.columns = ['number','T_Re', 'T_Se', 'c_Re',
       'f_H2', 'Sub', 'Fractal']
df_rese2.iloc[:,-1] = df_rese2.iloc[:,-1] *1

print(df_rese2)

In [ ]:
## Set the variable space and step size of the CVD experiment.
T_Re_min, T_Re_max, T_Re_step = [580, 680, 10] ## 11 steps
T_Re_var = np.arange(T_Re_min, T_Re_max+T_Re_step, T_Re_step)
T_Re_num = len(T_Re_var)
print(T_Re_var)
print(T_Re_num)

T_Se_min, T_Se_max, T_Se_step = [220, 300, 10] ## 9 steps
T_Se_var = np.arange(T_Se_min, T_Se_max+T_Se_step, T_Se_step)
T_Se_num = len(T_Se_var)
print(T_Se_num)

c_Re_min, c_Re_max, c_Re_step = [0.025, 0.15, 0.025] ## 6 steps
c_Re_var = np.arange(c_Re_min, c_Re_max+c_Re_step, c_Re_step) 
c_Re_num = len(c_Re_var)
print(c_Re_num)

f_H2_min, f_H2_max, f_H2_step = [0.01, 0.04, 0.01] ## 4 steps
f_H2_var = np.arange(f_H2_min, f_H2_max+f_H2_step, f_H2_step)
f_H2_num = len(f_H2_var)
print(f_H2_num)

Sub_min,Sub_max, Sub_step = [0, 1, 1] ## 2 steps
Sub_var = np.arange(Sub_min, Sub_max+Sub_step, Sub_step)
Sub_num = len(Sub_var)
print(Sub_num)



var_array = [T_Re_var, T_Se_var, 
             c_Re_var, f_H2_var, 
             Sub_var]
x_labels = ['T_Re', 'T_Se', 'c_Re',
       'f_H2', 'Sub']

def x_normalizer(X):
    
    def max_min_scaler(x, x_max, x_min):
        return (x-x_min)/(x_max-x_min)
    
    x_norm = []
    for x in (X):
           x_norm.append([max_min_scaler(x[i], 
                                         max(var_array[i]), 
                                         min(var_array[i])) for i in range(len(x))])  
    return np.array(x_norm)

def x_denormalizer(x_norm):
    def max_min_rescaler(x, x_max, x_min):
        return x*(x_max-x_min)+x_min
    
    x_original = []
    for x in (x_norm):
           x_original.append([max_min_rescaler(x[i], 
                                         max(var_array[i]), 
                                         min(var_array[i])) for i in range(len(x))])
    return np.array(x_original)



def get_closest_array(suggested_x):
    
    def get_closest_value(given_value, array_list):
        absolute_difference_function = lambda list_value : abs(list_value - given_value)
        closest_value = min(array_list, key=absolute_difference_function)
        return closest_value
    
    var_list = var_array
    modified_array = []
    for x in suggested_x:
        modified_array.append([get_closest_value(x[i], var_list[i]) for i in range(len(x))])
    return np.array(modified_array)

In [ ]:
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter

## Set the range of the parameter space after normalization
parameter_space = ParameterSpace([ContinuousParameter('T_Re', 0, 1),
                                 ContinuousParameter('T_Se', 0, 1),
                                 ContinuousParameter('c', 0, 1),
                                 ContinuousParameter('H2', 0, 1),
                                 DiscreteParameter('Sub', np.linspace(0,1,2)),
                                 ])
print(parameter_space)

In [ ]:
from typing import Union
from emukit.core.acquisition import Acquisition
from emukit.core.interfaces import IModel, IDifferentiable
from emukit.core.loop import FixedIntervalUpdater, OuterLoop, SequentialPointCalculator
from emukit.core.loop.loop_state import create_loop_state
from emukit.core.optimization import GradientAcquisitionOptimizer
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.core.acquisition import IntegratedHyperParameterAcquisition
from emukit.bayesian_optimization.local_penalization_calculator import LocalPenalizationPointCalculator

# create a loop for building Bayesian Optimization
class ProbabilisticBayesianOptimizationLoop(OuterLoop):
    def __init__(self, space: ParameterSpace, model_objective: Union[IModel, IDifferentiable],
                 acquisition: Acquisition = None,
                 update_interval: int = 1, batch_size: int = 1):

        self.model_objective = model_objective
        
        if acquisition is None:
            acquisition = ExpectedImprovement(model_objective)

        model_updater_objective = FixedIntervalUpdater(model_objective, update_interval)

        acquisition_optimizer = GradientAcquisitionOptimizer(space)
        if batch_size == 1:
            candidate_point_calculator = SequentialPointCalculator(acquisition, acquisition_optimizer)
        else:
            candidate_point_calculator = LocalPenalizationPointCalculator(acquisition, acquisition_optimizer,
                                                                          model_objective, space, batch_size)
        loop_state = create_loop_state(model_objective.X, model_objective.Y)

        super(ProbabilisticBayesianOptimizationLoop, self).__init__(candidate_point_calculator,
                                                                   [model_updater_objective],
                                                                   loop_state)


### Run GP Regression on the collected data

In [ ]:
# np.random.seed(20)

from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
def y_normalizer(y_2d):
     y_normalized=y_2d-1
     return y_normalized

def y_denormalizer(y_normalized):
     y_original=y_normalized+1
     return y_original
x_init = x_normalizer(df_rese2.iloc[:,1:6].values)
y_init = y_normalizer(np.transpose([df_rese2.iloc[:,-1].values]))
X, Y = [x_init, y_init]
print(X)
print(Y)


input_dim = len(X[0])
print(input_dim)
ker = GPy.kern.Matern32(input_dim = input_dim, ARD =True)#
ker.lengthscale.constrain_bounded(1e-1, 10)
ker.variance.constrain_bounded(1, 1000.0)

#ker += GPy.kern.Bias(input_dim = input_dim)
model_gpy = GPRegression(X , -Y, ker)#Emukit is a minimization tool; need to make Y negative
model_gpy.Gaussian_noise.variance = 0.05**2
model_gpy.Gaussian_noise.variance.fix()
model_gpy.randomize()
model_gpy.optimize_restarts(num_restarts=20,verbose =False, messages=False)
objective_model = GPyModelWrapper(model_gpy)



##### Check the performance of the regression model

In [ ]:
f_obj = objective_model.model.predict

y_pred, y_uncer = f_obj(X)
y_pred = -y_pred[:, -1]
y_uncer = np.sqrt(y_uncer[:, -1])
y_pred_unscaled = y_denormalizer(y_pred)

print(y_pred_unscaled)
print(y_uncer)

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(5, 3.5))  
fs = 18
lims1 = (1, 2)


ax.scatter(y_denormalizer(Y[:, -1]), y_pred_unscaled, alpha=0.5, 
           c=(112/255, 161/255, 255/255), edgecolor='navy')
ax.errorbar(y_denormalizer(Y[:, -1]), y_pred_unscaled, yerr=y_uncer, 
            ms=0, ls='', capsize=2, alpha=0.6, color='gray', zorder=0)
ax.plot(lims1, lims1, 'k--', alpha=0.75, zorder=0)

rmse_value = np.sqrt(mean_squared_error(Y[:, -1], y_pred))

ax.set_xlabel('True Values', fontsize=fs)
ax.set_ylabel('Predicted Values', fontsize=fs)
ax.tick_params(labelsize=fs-2)
ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Define new parameters for which predictions are needed
new_params = np.array([[580, 220, 0.075, 0.02, 1]])

# Use the model to predict new parameters, obtaining both the prediction values and uncertainty estimates
new_pred, new_uncer = objective_model.model.predict(x_normalizer(new_params))

new_pred = -new_pred[:, -1]
new_pred_unscaled = y_denormalizer(new_pred)

new_uncer = np.sqrt(new_uncer[:, -1])

print("Prediction (after denormalization):", new_pred_unscaled)
print("Uncertainty Estimate:", new_uncer)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.stats import spearmanr

mse = mean_squared_error
mse_all = mse(Y[:,-1], y_pred)
print ('all RMSE: %.4f' % (np.sqrt(mse_all)))

rsquared_all = r2_score(Y[:,-1], y_pred)
print ('all R^2: %.4f' % (rsquared_all))

sprman_all = spearmanr(Y[:,-1], y_pred)
print ('all spearman: %.4f' % (sprman_all[0]))

### Start the First Run of the Batch-mode Bayesian Optimization 

In [ ]:
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement, \
                                                      NegativeLowerConfidenceBound, \
                                                      MaxValueEntropySearch, \
                                                      MultipointExpectedImprovement,\
                                                      ProbabilityOfFeasibility, \
                                                      ProbabilityOfImprovement
# Expeceted Improvement (EI)
# acquisition = ExpectedImprovement(objective_model, jitter=0.05)
# acquisition_generator = lambda m: ExpectedImprovement(m, jitter=0.2)
# acquisition_integrated = IntegratedHyperParameterAcquisition(objective_model, acquisition_generator)
# acquisition = acquisition_integrated

## Lower Confidence Bound (LCB)
# acquisition = NegativeLowerConfidenceBound(objective_model, beta = 10)
## Probability Improvement (PI)
# acquisition = ProbabilityOfImprovement(objective_model,jitter=0)



# MaxValueEntropySearch (MES)
acquisition_generator = lambda m: MaxValueEntropySearch(m, parameter_space)
acquisition_integrated = IntegratedHyperParameterAcquisition(objective_model, acquisition_generator)
acquisition = acquisition_integrated

# Create a Bayesian optimization loop and collect new sample points
bayesopt_cons_pr = ProbabilisticBayesianOptimizationLoop(
    model_objective=objective_model,
    space=parameter_space,
    acquisition=acquisition,
    batch_size=15  # batchsize>10 to account for duplication
)

X_new = bayesopt_cons_pr.candidate_point_calculator.compute_next_points(bayesopt_cons_pr.loop_state)
X_new1= x_denormalizer(X_new)

f_obj = objective_model.model.predict

y_pred_new, y_uncer_new = f_obj(X_new)
y_pred_new = -y_pred_new
y_uncer_new = np.sqrt(y_uncer_new)
print(y_pred_new)


df_Xnew = pd.DataFrame(get_closest_array(X_new1), columns=df_rese2.columns[1:6])
df_all = pd.concat([df_rese2.iloc[:, 1:6], df_Xnew])
df_all_ = df_all.drop_duplicates()
df_Xnew = df_Xnew.sort_values(by=list(df_rese2.columns[1:6]), ignore_index = True)
df_Xnew.index = np.arange(len(df_Xnew))+len(df_rese2)
df_Xnew.iloc[:,:]



In [ ]:
df_Xnew_array = df_Xnew.iloc[:, :].values
result_array = get_closest_array(df_Xnew_array)

print(result_array)


In [ ]:
# Save the DataFrame to an Excel file
import pandas as pd

# Surragate Model
y_pred_new, y_uncer_new = f_obj(x_normalizer(result_array))
y_pred_new = -y_pred_new
y_uncer_new = np.sqrt(y_uncer_new)
# Aquisition function
f_acq = bayesopt_cons_pr.candidate_point_calculator.acquisition.evaluate 
acq_produc = f_acq(x_normalizer(result_array)) 
print(y_denormalizer(y_pred_new))
print(acq_produc)


X_new_columns = [f'X_new_{i}' for i in range(df_Xnew.shape[1])]
X_new_data = {col: result_array[:, i] for i, col in enumerate(X_new_columns)}
df_X_new = pd.DataFrame(X_new_data)
df_X_new['y_pred_new'] =y_denormalizer(y_pred_new) 
df_X_new['y_uncer_new'] = y_uncer_new
df_X_new['acq_produc'] = acq_produc


output_filename = "Next_round_suggestion.xlsx"
df_X_new.to_excel(output_filename, index=False)
print(f"The data has been successfully saved to an Excel file {output_filename}")



# Visualisation of Optimization Process

In [ ]:
file_path = 'Dataset_bayesian.xlsx'  # 修改为你的Excel文件路径
df_sug = pd.read_excel(file_path, sheet_name='sug-r3')


X_new = x_normalizer(df_sug.iloc[:, :5].values)

# 打印结果验证
print("X_new变量内容（前5行）：")
# 因为X_new是数组，我们打印前5行
print(X_new[:5])

# 获取列名（从原始DataFrame中获取前五列的列名）
column_names = list(df_sug.columns[:5])
print("\n前五列列名：", column_names)

# Calculate the data of the 3D CVD parameter-DF plot

In [ ]:
import numpy as np
from emukit.core.initial_designs.random_design import RandomDesign
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd

cmap_colors = [
    (30/255, 136/255, 229/255),   # Blue
    (1.0, 1.0, 1.0),             # White
    (255/255, 13/255, 87/255)    # Pink 
]

# Create color map
cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", cmap_colors)

np.random.seed(28)
design = RandomDesign(parameter_space)
x_sampled = design.get_samples(600)
x_columns = df_rese2.iloc[:,1:6].columns
x_sampled_df = pd.DataFrame(x_denormalizer(x_sampled), columns=x_columns)

def x_normalizer_for_z(z_values, z_min, z_max):
    """
    Normalize given z values.
    
    Parameters:
    - z_values: The original z value or array of z values to normalize.
    - z_min: Normalization parameter specifying the minimum z value.
    - z_max: Normalization parameter specifying the maximum z value.
    
    Returns:
    - Normalized z value or array of z values.
    """
    return (np.array(z_values) - z_min) / (z_max - z_min)

# Assume that the z-axis corresponds to T_Re_var, i.e., the range of z-axis values is [T_Re_min, T_Re_max]


def create_3d_stacked_heatmaps(dim1, dim2, dim3, metric_type='mean', uncertainty=False, z_values=[0.05,0.1,0.15], n_grid=21):
    print("\n[Info] Generating 3D stacked heatmap data...")
    # Use provided z values instead of uniform distribution
    z_levels = z_values
    heatmap_layers = []
    z_positions_original = []

    for z_idx, z_val in enumerate(z_levels):
        print(f"  --> Processing layer {z_idx+1}/{len(z_levels)}... ")
        x1x2y_data = []

        # Normalize z value
        z_normalized = x_normalizer_for_z(np.array([z_val]), T_Re_min, T_Re_max)[0]

        for x1 in np.linspace(0, 1, n_grid):
            for x2 in np.linspace(0, 1, n_grid):
                x_base = np.mean(x_sampled, axis=0)
                x_base[dim1] = x1
                x_base[dim2] = x2
                x_base[dim3] = z_normalized # Use normalized z value
                x_input = x_base.reshape(1, -1)

                y_pred, y_uncer = f_obj(x_input)
                y_pred = -y_pred + 1

                if uncertainty:
                    uncer_val = np.sqrt(np.abs(y_uncer[0]))  # Avoid sqrt(negative) causing NaN
                    val = uncer_val if not np.isnan(uncer_val) else 0.0
                else:
                    if metric_type == 'max':
                        val = np.max(y_pred)
                    elif metric_type == 'min':
                        val = np.min(y_pred)
                    else:
                        val = np.mean(y_pred)

                x_org = x_denormalizer(x_input)[0]
                x1x2y_data.append([x_org[dim1], x_org[dim2], val])

        data_array = np.array(x1x2y_data, dtype=float)
        x1_grid = data_array[:, 0].reshape(n_grid, n_grid)
        x2_grid = data_array[:, 1].reshape(n_grid, n_grid)
        y_grid = data_array[:, 2].reshape(n_grid, n_grid)
        heatmap_layers.append((x1_grid, x2_grid, y_grid))

        x_for_z = np.mean(x_sampled, axis=0)
        x_for_z[dim3] = z_val
        
        z_positions_original.append(z_val)
    
    # 3D plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Color bar
    vmin, vmax = (0.05, 0.10) if uncertainty else (1.1, 1.8)
    title_prefix = "Uncertainty" if uncertainty else "Objective Function"

    for i, (x1_grid, x2_grid, y_grid) in enumerate(heatmap_layers):
        z_grid = np.full_like(x1_grid, z_positions_original[i])
        normalized = (y_grid - vmin) / (vmax - vmin)
        normalized = np.clip(normalized, 0, 1).astype(float)
        facecolors = cmap(normalized)

        # Heatmap settings
        ax.plot_surface(
            x1_grid, x2_grid, z_grid,
            facecolors=facecolors,
            alpha=1.0, linewidth=0.0, antialiased=True, shade=False
        )
        ax.contour(x1_grid, x2_grid, z_grid, levels=[z_positions_original[i]],
                   colors='black', alpha=0.8, linewidths=2.0)

    X_orig = x_denormalizer(X)
    ax.scatter(X_orig[:, dim1], X_orig[:, dim2], X_orig[:, dim3],
               c='gray', s=250, alpha=0.9, edgecolors='black', linewidth=1, label='Original samples')

    X_new_orig = x_denormalizer(X_new)
    ax.scatter(X_new_orig[:, dim1], X_new_orig[:, dim2], X_new_orig[:, dim3],
               c='black', s=300, alpha=1.0, edgecolors='darkgreen', linewidth=2,
               marker='*', label='New samples')

    ax.set_xlabel(f'{x_columns[dim1]}', fontsize=14)
    ax.set_ylabel(f'{x_columns[dim2]}', fontsize=14)
    ax.set_zlabel(f'{x_columns[dim3]}', fontsize=14)
    ax.set_title(f'3D Stacked Heatmaps: {title_prefix} ({metric_type})', fontsize=16, pad=20)

    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, shrink=0.6, aspect=20, pad=0.1)
    cbar.set_label(title_prefix, fontsize=12)
    cbar.ax.tick_params(labelsize=10)

    ax.legend(loc='upper right', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.view_init(elev=10, azim=30)

    plt.tight_layout()
    plt.show()

    # Create a Pandas Excel writer using xlsxwriter as the engine
    output_excel = f"heatmap_data_dim_{dim1}_{dim2}_{dim3}.xlsx"
    
    with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
        for i, (x1_grid, x2_grid, y_grid) in enumerate(heatmap_layers):
            z_val = z_positions_original[i]
            print(f"--> Saving data for z={z_val} to Excel...")
            
            # Flatten grid data
            data_flat = np.column_stack((
                x1_grid.ravel(), 
                x2_grid.ravel(), 
                y_grid.ravel()
            ))
            
            df = pd.DataFrame(data_flat, columns=[f'Dim{dim1}', f'Dim{dim2}', 'Value'])
            
            # Write to different sheets in Excel
            sheet_name = f'z_{int(z_val)}'
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f"\n✅ Data successfully saved to {output_excel}")

    return fig, ax, heatmap_layers, z_positions_original


# Example usage
if __name__ == "__main__":
    # Select three feature dimensions for visualization
    feature_dims = [1, 2, 0]  # Adjust according to your needs
    
    # print("Creating 3D stacked heatmap visualization...")
    fig1, ax1, layers1, z_pos1 = create_3d_stacked_heatmaps(
        feature_dims[0], feature_dims[1], feature_dims[2], 
        metric_type='max', uncertainty=False, z_values=[580, 620, 660], n_grid=21
    )

    # Also show uncertainty
    print("Creating 3D stacked heatmap of uncertainty...")
    fig3, ax3, layers3, z_pos3 = create_3d_stacked_heatmaps(
        feature_dims[0], feature_dims[1], feature_dims[2], 
        metric_type='max', uncertainty=True, z_values=[580, 620, 660], n_grid=21
    )

# Calculate the data of the 2D CVD parameter-DF plot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from emukit.core.initial_designs.random_design import RandomDesign
import os

# Set the output path
output_folder = "C:\\Users\\黄文强\\Desktop\\project\\ML4dentrites\\1-Process optimization\\process_data_r1"
os.makedirs(output_folder, exist_ok=True)

input_dim = 5  # Number of input dimensions, e.g., 5 features

# Create color maps
mean_cmap_colors = [
    (30/255, 136/255, 229/255),   # Blue
    (1.0, 1.0, 1.0),             # White
    (255/255, 13/255, 87/255)    # Pink 
]
mean_cmap = mcolors.LinearSegmentedColormap.from_list("mean_colormap", mean_cmap_colors, N=256)

# Three-color gradient for uncertainty heatmaps: dark blue → white → red
uncertainty_cmap_colors = [
    (0/255, 0/255, 255/255),     # Dark blue
    (1.0, 1.0, 1.0),             # White
    (255/255, 0/255, 0/255)      # Red
]
uncertainty_cmap = mcolors.LinearSegmentedColormap.from_list("uncertainty_colormap", uncertainty_cmap_colors, N=256)

mean_levels = np.linspace(1.1,1.8,42)
uncertainty_levels = np.linspace(0.05, 0.15, 60)

# Set random seed and generate samples
np.random.seed(10)
design = RandomDesign(parameter_space)
x_sampled = design.get_samples(200)
x_columns = df_rese2.iloc[:,1:6].columns
x_sampled_df = pd.DataFrame(x_denormalizer(x_sampled), columns=x_columns)

for i in range(input_dim):
    for j in range(input_dim - i - 1):
        ind1 = i
        ind2 = j + i + 1
        
        n_steps = 21
        x1x2y_pred = []
        x1x2y_uncer = []

        # Build grid and calculate predictions and uncertainties
        for x1 in np.linspace(0, 1, n_steps):
            for x2 in np.linspace(0, 1, n_steps):
                x_temp = np.copy(x_sampled)
                x_temp[:, ind1] = x1
                x_temp[:, ind2] = x2
                y_pred, y_uncer = f_obj(x_temp)
                y_pred = -y_pred + 1  # Optional: Adjust prediction values direction
                
                x1_org = x_denormalizer(x_temp)[0, ind1]
                x2_org = x_denormalizer(x_temp)[0, ind2]

                # Collect prediction statistics
                x1x2y_pred.append([
                    x1_org, x2_org,
                    np.max(y_pred),
                    np.mean(y_pred),
                    np.min(y_pred)
                ])

                # Collect uncertainty statistics
                x1x2y_uncer.append([
                    x1_org, x2_org,
                    np.max(np.sqrt(y_uncer)),
                    np.mean(np.sqrt(y_uncer)),
                    np.min(np.sqrt(y_uncer))
                ])
        
        # Convert to numpy arrays and reshape
        x1 = np.array([row[0] for row in x1x2y_pred], dtype=float).reshape(n_steps, n_steps)
        x2 = np.array([row[1] for row in x1x2y_pred], dtype=float).reshape(n_steps, n_steps)

        y_pred_max = np.array([row[2] for row in x1x2y_pred], dtype=float).reshape(n_steps, n_steps)
        y_pred_mean = np.array([row[3] for row in x1x2y_pred], dtype=float).reshape(n_steps, n_steps)
        y_pred_min = np.array([row[4] for row in x1x2y_pred], dtype=float).reshape(n_steps, n_steps)

        y_uncer_max = np.array([row[2] for row in x1x2y_uncer], dtype=float).reshape(n_steps, n_steps)
        y_uncer_mean = np.array([row[3] for row in x1x2y_uncer], dtype=float).reshape(n_steps, n_steps)
        y_uncer_min = np.array([row[4] for row in x1x2y_uncer], dtype=float).reshape(n_steps, n_steps)

        fs = 20
        title_pad = 16

        # Plot Prediction figures
        fig, axes = plt.subplots(1, 3, figsize=(17, 3.5), sharey=False, sharex=False)
        for ax, y, title in zip(axes, [y_pred_max, y_pred_mean, y_pred_min],
                                ['Max Prediction', 'Mean Prediction', 'Min Prediction']):
            c_plt1 = ax.contourf(x1, x2, y, levels=mean_levels, cmap=mean_cmap, extend='both')
            cbar = fig.colorbar(c_plt1, ax=ax)
            cbar.ax.tick_params(labelsize=fs*0.8)

            ax.scatter(x_denormalizer(X)[:, ind1], x_denormalizer(X)[:, ind2],
                       s=80, facecolors='gray', alpha=0.9, edgecolor='gray')
            ax.scatter(x_denormalizer(X_new)[:, ind1], x_denormalizer(X_new)[:, ind2],
                       s=100, facecolors='orange', alpha=0.9, edgecolor='orange')

            ax.set_xlabel(str(x_columns[ind1]), fontsize=fs)
            ax.set_ylabel(str(x_columns[ind2]), fontsize=fs)

            x1_delta = (np.max(x1)-np.min(x1)) * 0.04
            x2_delta = (np.max(x2)-np.min(x2)) * 0.04
            ax.set_xlim(np.min(x1)-x1_delta, np.max(x1)+x1_delta)
            ax.set_ylim(np.min(x2)-x2_delta, np.max(x2)+x2_delta)
            ax.tick_params(direction='in', length=5, width=1, labelsize=fs*0.8)

            if ind1 == 0:
                ax.set_xticks([580, 610, 640, 670])
            if ind1 == 1:
                ax.set_xticks([220, 240, 260, 280, 300])
            if ind1 == 2:
                ax.set_xticks([0.05, 0.1, 0.15])
            if ind2 == 4:
                ax.set_yticks([0, 1])

        axes[0].set_title('Objective function max', pad=title_pad, fontsize=fs)
        axes[1].set_title('Objective function mean', pad=title_pad, fontsize=fs)
        axes[2].set_title('Objective function min', pad=title_pad, fontsize=fs)
        plt.subplots_adjust(wspace=0.3)

        # Save Prediction figures
        feature1 = x_columns[ind1]
        feature2 = x_columns[ind2]
        pred_image_path = os.path.join(output_folder, f"{feature1}_vs_{feature2}_prediction.png")
        plt.savefig(pred_image_path, dpi=600, bbox_inches='tight')  # High resolution, tight margins
        plt.close(fig)  # Close current figure to save memory

        # Plot Uncertainty figures
        fig, axes = plt.subplots(1, 3, figsize=(17, 3.5), sharey=False, sharex=False)
        for ax, y, title in zip(axes, [y_uncer_max, y_uncer_mean, y_uncer_min],
                                ['Max Uncertainty', 'Mean Uncertainty', 'Min Uncertainty']):
            c_plt1 = ax.contourf(x1, x2, y, levels=uncertainty_levels, cmap=uncertainty_cmap, extend='both')
            cbar = fig.colorbar(c_plt1, ax=ax)
            cbar.ax.tick_params(labelsize=fs*0.8)

            ax.scatter(x_denormalizer(X)[:, ind1], x_denormalizer(X)[:, ind2],
                       s=80, facecolors='gray', alpha=0.9, edgecolor='gray')
            ax.scatter(x_denormalizer(X_new)[:, ind1], x_denormalizer(X_new)[:, ind2],
                       s=100, facecolors='orange', alpha=0.9, edgecolor='orange')

            ax.set_xlabel(str(x_columns[ind1]), fontsize=fs)
            ax.set_ylabel(str(x_columns[ind2]), fontsize=fs)

            x1_delta = (np.max(x1)-np.min(x1)) * 0.04
            x2_delta = (np.max(x2)-np.min(x2)) * 0.04
            ax.set_xlim(np.min(x1)-x1_delta, np.max(x1)+x1_delta)
            ax.set_ylim(np.min(x2)-x2_delta, np.max(x2)+x2_delta)
            ax.tick_params(direction='in', length=5, width=1, labelsize=fs*0.8)

            if ind1 == 0:
                ax.set_xticks([580, 610, 640, 670])
            if ind1 == 1:
                ax.set_xticks([220, 240, 260, 280, 300])
            if ind1 == 2:
                ax.set_xticks([0.05, 0.1, 0.15])
            if ind2 == 4:
                ax.set_yticks([0, 1])

        axes[0].set_title('Objective uncertainty max', pad=title_pad, fontsize=fs)
        axes[1].set_title('Objective uncertainty mean', pad=title_pad, fontsize=fs)
        axes[2].set_title('Objective uncertainty min', pad=title_pad, fontsize=fs)
        plt.subplots_adjust(wspace=0.3)

        # Save Uncertainty figures
        uncer_image_path = os.path.join(output_folder, f"{feature1}_vs_{feature2}_uncertainty.png")
        plt.savefig(uncer_image_path, dpi=600, bbox_inches='tight')
        plt.close(fig)

        # Construct DataFrame and save as Excel
        pred_df = pd.DataFrame(x1x2y_pred, columns=[feature1, feature2, 'Max Prediction', 'Mean Prediction', 'Min Prediction'])
        uncer_df = pd.DataFrame(x1x2y_uncer, columns=[feature1, feature2, 'Max Uncertainty', 'Mean Uncertainty', 'Min Uncertainty'])

        with pd.ExcelWriter(os.path.join(output_folder, f"{feature1}_vs_{feature2}.xlsx")) as writer:
            pred_df.to_excel(writer, sheet_name='Prediction', index=False)
            uncer_df.to_excel(writer, sheet_name='Uncertainty', index=False)

        print(f"📊 Saved {feature1}_vs_{feature2}.xlsx to {output_folder}")
        print(f"🖼️ Saved images to: {pred_image_path} and {uncer_image_path}")